## Imports

In [147]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point

## Useful functions

In [148]:
from scipy import stats
from typing import Tuple


def save_df(df: pd.DataFrame, name: str, directory: str, message=None) -> None:
    """
    Function for saving DataFrame to .csv file in the given directory with given name

        Args:
            df (pd.DataFrame): DataFrame to be saved
            name (str): Name of file
            directory (str): Name of the directory where the file is being saved
            message (str): Message to be printed

        Returns:
            None
    """
    df.to_csv(directory + "/" + name, encoding="utf-8")
    if message is None:
        print(f"{name} saved in {directory}/")
    else:
        print(message)


def calculate_SPI(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function to calculate Standardized Precipitation Index (SPI) for the given DataFrame.
    The function fits a gamma distribution to the precipitation data and then transforms the
    cumulative distribution function (CDF) of the gamma distribution to calculate the SPI values.
    The returned DataFrame has the same index as the input DataFrame, with each date associated
    with its corresponding SPI value.

    Args:
    df (pd.DataFrame): DataFrame containing the '24h_precipitation_mm' column with precipitation data.

    Returns:
    pd.DataFrame: DataFrame containing SPI for the given data.
    """
    precip_sum = df["24h_precipitation_mm"]
    precip_sum[precip_sum <= 0] = 1e-15  # !!!!!!!!!
    params = stats.gamma.fit(precip_sum, floc=0)
    shape, loc, scale = params
    cdf = stats.gamma.cdf(precip_sum, shape, loc, scale)
    SPI = stats.norm.ppf(cdf)
    return pd.DataFrame({"SPI": SPI}, index=df.index)


def get_SPI(
    df: pd.DataFrame, voi: str, save=True
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Function to calculate SPI for different periods (SPI-1, SPI-3, SPI-12) and save them to CSV files.

    Args:
    df (pd.DataFrame): DataFrame containing the '24h_precipitation_mm' column with precipitation data.
    voi (str): Name of the analyzed voivodeship.
    save (bool): Flags whether to save the SPI results.

    Returns:
    tuple: Tuple containing SPI-1, SPI-3, and SPI-12 as DataFrames.
    """

    print("Calculating SPI...")

    df.index = pd.to_datetime(df.index)

    SPI_1Y = df.resample("YE").agg({"24h_precipitation_mm": "sum"}).dropna()
    SPI_1 = calculate_SPI(SPI_1Y)

    SPI_3M = df.resample("QE").agg({"24h_precipitation_mm": "sum"}).dropna()
    SPI_3 = calculate_SPI(SPI_3M)

    SPI_1M = df.resample("ME").agg({"24h_precipitation_mm": "sum"}).dropna()
    SPI_12 = calculate_SPI(SPI_1M)

    if save:
        save_df(SPI_1, f"{voi}_SPI_yearly.csv", "../results")
        save_df(SPI_3, f"{voi}_SPI_quarterly.csv", "../results")
        save_df(SPI_12, f"{voi}_SPI_monthly.csv", "../results")

    print("SPI calculated.")

    return SPI_1, SPI_3, SPI_12

In [149]:
import geopandas as gpd


def clip_precip_to_voi(precip: pd.DataFrame, voi_gdf: gpd.GeoDataFrame) -> pd.DataFrame:
    """Function to clip precipitation data to only one voivodeship

    Args:
        precip (pd.DataFrame): Data containing precipitation over years
        voi_gdf (gpd.GeoDataFrame): GeoDataFrame containing details about stations from given voivodeship

    Returns:
        pd.DataFrame: DataFrame containing precipitation data merged with stations'
                      detail from one voivodeship
    """

    merged_df = precip.merge(
        voi_gdf, how="inner", left_on="station_code", right_on="ID"
    )
    merged_df = merged_df[
        [
            "station_code",
            "station_name",
            "year",
            "month",
            "day",
            "24h_precipitation_mm",
            "SMDB_status",
            "precip_type",
            "snow_cover_cm",
            "river",
            "lat",
            "lon",
            "altitude",
        ]
    ]

    return merged_df


def clip_to_voivodeship(
    gdf: gpd.GeoDataFrame, geojson: gpd.GeoDataFrame, voi: str
) -> Tuple[gpd.GeoSeries, gpd.GeoDataFrame]:
    """Function to clip GeoDataFrame to specific voivodeship borders

    Args:
        gdf (gpd.GeoDataFrame): GeoDataFrame containing stations data
        geojson (gpd.GeoDataFrame): GeoDataFrame containing voivodeship borders
        voi (str): Name of the voivodeship to clip the data to

    Returns:
        Tuple[gpd.GeoSeries, gpd.GeoDataFrame]: Tuple containing voivodeship polygon
                                                and clipped GeoDataFrame
    """
    voi_polygon = geojson[geojson["name"] == voi]["geometry"]
    voi_gdf = gdf[gdf.within(voi_polygon.geometry.iloc[0])]
    return voi_polygon, voi_gdf


def get_voivodeship_borders() -> gpd.GeoDataFrame:
    """Function to fetch and return voivodeship borders data as GeoDataFrame from
       a specified URL with spaces and dashes removed from voivodeship names.

    Returns:
        gpd.GeoDataFrame: GeoDataFrame containing voivodeship borders
    """
    geojson = gpd.read_file(
        "https://simplemaps.com/static/svg/country/pl/admin1/pl.json"
    )
    geojson["name"] = (
        geojson["name"]
        .apply(lambda x: x.replace(" ", ""))
        .apply(lambda x: x.replace("-", ""))
    )
    return geojson


def clip_data_to_voi(stations: gpd.GeoDataFrame, voi: str) -> gpd.GeoDataFrame:
    """Pipeline for clipping all the data to voivodeships

    Args:
        precip (pd.DataFrame): All precipitation data
        stations (gpd.GeoDataFrame): All stations data
        voi (str): Voivodeship name

    Returns:
        list[gpd.GeoDataFrame, gpd.GeoDataFrame, gpd.GeoDataFrame]: Polygon of voivodeship, precipitation data clipped
          to voivodeship & stations clipped to voivodeship
    """
    print(f"Clipping precipitation and stations data to {voi} voivodeship...")
    geojson = get_voivodeship_borders()
    voi_polygon, voi_stations_gdf = clip_to_voivodeship(stations, geojson, voi)
    # voi_precip_gdf = clip_precip_to_voi(precip, voi_stations_gdf)

    print("Clipping ended")

    return voi_polygon  # , voi_precip_gdf#, voi_stations_gdf

## Loading data

In [150]:
preprocessed_voi_df = pd.read_csv(
    "../data/preprocessed_Lubusz_data.csv", index_col="date"
)
preprocessed_voi_df

,station_code,station_name,24h_precipitation_mm,SMDB_status,precip_type,snow_cover_cm,river,lat,lon,altitude
date,,,,,,,,,,
1991-01-01,251150110,ŻAGAŃ,1.9,Normal,Water,0.0,Bóbr,51.649722,15.293611,96.0
1991-01-02,251150110,ŻAGAŃ,2.5,Normal,Water,0.0,Bóbr,51.649722,15.293611,96.0
1991-01-03,251150110,ŻAGAŃ,1.1,Normal,Water,0.0,Bóbr,51.649722,15.293611,96.0
1991-01-04,251150110,ŻAGAŃ,3.2,Normal,Water,0.0,Bóbr,51.649722,15.293611,96.0
1991-01-05,251150110,ŻAGAŃ,1.3,Normal,Water,0.0,Bóbr,51.649722,15.293611,96.0
...,...,...,...,...,...,...,...,...,...,...
2022-12-24,252150180,LUBINICKO-ŚWIEBODZIN,2.5,Normal,Water,0.0,Obrzyca,52.242778,15.545278,88.0
2022-12-25,252150180,LUBINICKO-ŚWIEBODZIN,8.6,Normal,Water,0.0,Obrzyca,52.242778,15.545278,88.0
2022-12-26,252150180,LUBINICKO-ŚWIEBODZIN,0.5,Normal,Water,0.0,Obrzyca,52.242778,15.545278,88.0


In [151]:
stations_gdf = gpd.read_file("../data/stations.shp", encoding="cp1250")
voi_polygon = clip_data_to_voi(stations_gdf, "Lubusz")
voi_polygon

Clipping precipitation and stations data to Lubusz voivodeship...
Clipping ended


7    POLYGON ((16.37113 51.74919, 16.32214 51.69995...
Name: geometry, dtype: geometry

## SPIs analysis for an each station from voivodeship (Lubusz)

In [152]:
def get_stations_SPI_statistics(SPI_1, SPI_3, SPI_12, station_name, voi):
    save_df(
        pd.concat(
            [
                SPI_1.describe().rename(columns={"SPI": "Values"}),
                SPI_3.describe().rename(columns={"SPI": "Values"}),
                SPI_12.describe().rename(columns={"SPI": "Values"}),
            ],
            axis=1,
            keys=["SPI_1", "SPI_3", "SPI_12"],
        ),
        f"{station_name}-{voi}_SPI_statistics.csv",
        "../results",
    )
    print(
        f"Saved SPIs descriptive statistics from {station_name} station in the {voi} voivodeship in results/{station_name}-{voi}_SPI_statistics.csv"
    )

In [162]:
def map_to_range(value):
    SPI_ranges = {
        "Extremely wet": (2.0, float("inf")),
        "Very wet": (1.5, 1.99),
        "Moderately wet": (1.0, 1.49),
        "Moderate conditions": (-0.99, 0.99),
        "Moderate drought": (-1.49, -1.0),
        "Severe drought": (-1.99, -1.5),
        "Extreme drought": (-float("inf"), -2.0),
    }
    for range_name, (lower_bound, upper_bound) in SPI_ranges.items():
        if lower_bound <= value <= upper_bound:
            return f"{range_name}"

In [163]:
def visualize_stations_SPI(SPI_1, SPI_3, SPI_12, s, voi):
    # SPI_1["State"] = SPI_1.map(map_to_range)

    SPI_dict = {"SPI_1": SPI_1, "SPI_3": SPI_3, "SPI_12": SPI_12}

    color_ranges = {
        "darkblue": (2.0, float("inf")),
        "blue": (1.5, 1.99),
        "cornflowerblue": (1.0, 1.49),
        "mediumaquamarine": (-0.99, 0.99),
        "wheat": (-1.49, -1.0),
        "sandybrown": (-1.99, -1.5),
        "firebrick": (-float("inf"), -2.0),
    }

    color_mapping = {
        "darkblue": "Extremely wet",
        "blue": "Very wet",
        "cornflowerblue": "Moderately wet",
        "mediumaquamarine": "Moderate conditions",
        "wheat": "Moderate drought",
        "sandybrown": "Severe drought",
        "firebrick": "Extreme drought",
    }

    for spi_key, spi_val in SPI_dict.items():

        colors = []
        legend_dict = {}

        for value in spi_val["SPI"]:
            for range_name, (lower_bound, upper_bound) in color_ranges.items():
                if lower_bound <= value <= upper_bound:
                    colors.append(range_name)
                    legend_dict[range_name] = color_mapping[range_name]

        custom_order = color_mapping.keys()
        legend_dict_sorted = {
            key: legend_dict[key] for key in custom_order if key in legend_dict.keys()
        }

        plt.figure(figsize=(10, 6))
        plt.plot(
            spi_val.index, spi_val["SPI"], linestyle="-", color="darkgrey", alpha=0.7
        )

        for i in range(len(spi_val["SPI"]) - 1):
            plt.scatter(
                spi_val.index[i : i + 2],
                spi_val["SPI"][i : i + 2],
                marker="o",
                color=colors[i],
            )

        handles = [
            plt.Line2D(
                [0], [0], marker="o", color="w", label=label, markerfacecolor=color
            )
            for color, label in legend_dict_sorted.items()
        ]
        plt.legend(
            handles=handles,
            title="Precipitation conditions",
            loc="right",
            bbox_to_anchor=(0.77, 0.25, 0.5, 0.5),
        )
        plt.title(f"{spi_key} in time for station {s} in {voi} voivodeship")
        plt.xlabel("Date")
        plt.ylabel("SPI")
        plt.grid(True)
        plt.savefig(f"../results/{spi_key}_{s}-{voi}.png", bbox_inches="tight")
        print(
            f"Figure with {spi_key} for station {s} in {voi} voivodeship saved in results/{spi_key}_{s}-{voi}.png"
        )
        # plt.show()

In [164]:
def compare_stations_SPI(SPI_1, SPI_3, SPI_12, s, voi):
    SPI_1_resampled = SPI_1.resample("YE").agg({"SPI": "mean"})
    SPI_3_resampled = SPI_3.resample("YE").agg({"SPI": "mean"})

    fig, ax = plt.subplots(1, 1, figsize=(10, 6))

    sns.lineplot(
        SPI_1_resampled,
        x=SPI_1_resampled.index,
        y=SPI_1_resampled["SPI"],
        ax=ax,
        label="SPI_1",
        alpha=0.5,
    )
    sns.lineplot(
        SPI_3_resampled,
        x=SPI_3_resampled.index,
        y=SPI_3_resampled["SPI"],
        ax=ax,
        label="SPI_3",
        alpha=0.5,
    )
    sns.lineplot(
        SPI_12, x=SPI_12.index, y=SPI_12["SPI"], ax=ax, label="SPI_12", alpha=0.5
    )

    ax.set(
        title=f"Comparison of SPIs for stations {s} in {voi} voivodeship", xlabel="Year"
    )
    ax.grid(True)

    plt.savefig(f"../results/SPI_comparison_{s}-{voi}.png", bbox_inches="tight")
    print(
        f"Figure with SPIs comparison for station {s} in {voi} voivodeship saved in results/SPI_comparison_{s}-{voi}.png"
    )
    # plt.show()

In [165]:
def voi_SPI_map(avg_SPIs, voi_polygon, voi):
    geometry = [Point(lon, lat) for lon, lat in zip(avg_SPIs["lon"], avg_SPIs["lat"])]
    gdf = gpd.GeoDataFrame(avg_SPIs, geometry=geometry, crs="EPSG:4326")

    gdf["SPI_1_State"] = gdf["SPI_1"].map(map_to_range)
    gdf["SPI_3_State"] = gdf["SPI_3"].map(map_to_range)
    gdf["SPI_12_State"] = gdf["SPI_12"].map(map_to_range)

    fig, ax = plt.subplots(3, 1, figsize=(8, 30))

    x, y = voi_polygon.iloc[0].exterior.xy
    ax[0].plot(x, y, color="blue")
    ax[0].set_xlim([min(gdf.geometry.x) - 0.75, max(gdf.geometry.x) + 0.75])
    ax[0].set_ylim([min(gdf.geometry.y) - 0.75, max(gdf.geometry.y) + 0.75])
    sns.scatterplot(gdf, x=gdf.geometry.x, y=gdf.geometry.y, hue="SPI_1", ax=ax[0])
    ax[0].set(
        title=f"Mean SPI_1 in {voi} over time", xlabel="Longitude", ylabel="Latitude"
    )

    plt.show()

In [166]:
def stations_SPI_pipeline(preprocessed_voi_df, voi_polygon, voi):
    station_names = preprocessed_voi_df["station_name"].unique()
    avg_SPIs = pd.DataFrame(
        index=station_names, columns=["SPI_1", "SPI_3", "SPI_12", "lat", "lon"]
    )
    for s in station_names:
        SPI_1, SPI_3, SPI_12 = get_SPI(
            preprocessed_voi_df[preprocessed_voi_df["station_name"] == s],
            voi,
            False,
        )
        avg_SPIs.loc[s, "SPI_1"] = SPI_1["SPI"].mean()
        avg_SPIs.loc[s, "SPI_3"] = SPI_3["SPI"].mean()
        avg_SPIs.loc[s, "SPI_12"] = SPI_12["SPI"].mean()
        avg_SPIs.loc[s, "lat"] = preprocessed_voi_df[
            preprocessed_voi_df["station_name"] == s
        ]["lat"].iloc[0]
        avg_SPIs.loc[s, "lon"] = preprocessed_voi_df[
            preprocessed_voi_df["station_name"] == s
        ]["lon"].iloc[0]
        SPI_1["SPI"] = SPI_1["SPI"].round(2)
        SPI_3["SPI"] = SPI_3["SPI"].round(2)
        SPI_12["SPI"] = SPI_12["SPI"].round(2)
        # get_stations_SPI_statistics(SPI_1, SPI_3, SPI_12, s, voi)
        # visualize_stations_SPI(SPI_1, SPI_3, SPI_12, s, voi)
        # compare_stations_SPI(SPI_1, SPI_3, SPI_12, s, voi)
    voi_SPI_map(avg_SPIs, voi_polygon, voi)

In [ ]:
stations_SPI_pipeline(preprocessed_voi_df, voi_polygon, "Lubusz")

Calculating SPI...
SPI calculated.
Calculating SPI...
SPI calculated.
Calculating SPI...
SPI calculated.
Calculating SPI...
SPI calculated.
Calculating SPI...
SPI calculated.
Calculating SPI...
SPI calculated.
Calculating SPI...
SPI calculated.
                         SPI_1     SPI_3    SPI_12        lat        lon  \
ŻAGAŃ                 0.000426 -0.000805  0.003109  51.649722  15.293611   
BOCZÓW                0.000381 -0.001329  0.004336  52.324722  14.948889   
DREZDENKO             0.000988   0.00169  0.166543  52.854722  15.841111   
BABIMOST              0.000665 -0.001538   0.19547  52.143611  15.803333   
LUBINICKO-ŚWIEBODZIN   0.00048 -0.001775   0.00839  52.242778  15.545278   
KRZYŻ                -0.001843 -0.001953  0.009137  52.881111  15.983611   
SANICE                0.000636   0.00133  0.005057  51.406667  14.975556   

                                       geometry      SPI_1_State  
ŻAGAŃ                 POINT (15.29361 51.64972)  Średnie warunki  
BOCZÓW      

## SPI analysis for the voivodeship (Lubusz)

In [ ]:
def get_voi_SPI_statistics(SPI_1, SPI_3, SPI_12, voi):
    save_df(
        pd.concat(
            [
                SPI_1.describe().rename(columns={"SPI": "Values"}),
                SPI_3.describe().rename(columns={"SPI": "Values"}),
                SPI_12.describe().rename(columns={"SPI": "Values"}),
            ],
            axis=1,
            keys=["SPI_1", "SPI_3", "SPI_12"],
        ),
        f"{voi}_SPI_statistics.csv",
        "../results",
    )
    print(
        f"Saved SPIs descriptive statistics from {voi} voivodeship in results/{voi}_SPI_statistics.csv"
    )

In [ ]:
def visualize_voi_SPI(SPI_1, SPI_3, SPI_12, voi):
    SPI_dict = {"SPI_1": SPI_1, "SPI_3": SPI_3, "SPI_12": SPI_12}

    color_ranges = {
        "darkblue": (2.0, float("inf")),
        "blue": (1.5, 1.99),
        "cornflowerblue": (1.0, 1.49),
        "mediumaquamarine": (-0.99, 0.99),
        "wheat": (-1.49, -1.0),
        "sandybrown": (-1.99, -1.5),
        "firebrick": (-float("inf"), -2.0),
    }

    color_mapping = {
        "darkblue": "Extremely wet",
        "blue": "Very wet",
        "cornflowerblue": "Moderately wet",
        "mediumaquamarine": "Moderate conditions",
        "wheat": "Moderate drought",
        "sandybrown": "Severe drought",
        "firebrick": "Extreme drought",
    }

    fig, ax = plt.subplots(3, 1, figsize=(14, 17))
    n = 0

    for spi_key, spi_val in SPI_dict.items():

        colors = []
        legend_dict = {}

        for value in spi_val["SPI"]:
            for range_name, (lower_bound, upper_bound) in color_ranges.items():
                if lower_bound <= value <= upper_bound:
                    colors.append(range_name)
                    legend_dict[range_name] = color_mapping[range_name]

        custom_order = color_mapping.keys()
        legend_dict_sorted = {
            key: legend_dict[key] for key in custom_order if key in legend_dict.keys()
        }

        ax[n].plot(
            spi_val.index, spi_val["SPI"], linestyle="-", color="darkgrey", alpha=0.7
        )

        for i in range(len(spi_val["SPI"]) - 1):
            ax[n].scatter(
                spi_val.index[i : i + 2],
                spi_val["SPI"][i : i + 2],
                marker="o",
                color=colors[i],
            )

        handles = [
            plt.Line2D(
                [0], [0], marker="o", color="w", label=label, markerfacecolor=color
            )
            for color, label in legend_dict_sorted.items()
        ]
        ax[n].legend(
            handles=handles,
            title="Precipitation conditions",
            loc="right",
            bbox_to_anchor=(0.68, 0.25, 0.5, 0.5),
        )
        ax[n].set(
            title=f"{spi_key} in time for the {voi} voivodeship",
            xlabel="Date",
            ylabel="SPI",
        )
        ax[n].grid(True)
        n += 1

    plt.suptitle(f"SPIs in time for the {voi} voivodeship", y=1, fontsize=16)
    plt.tight_layout()
    plt.savefig(f"../results/SPI_in_time_{voi}.png", bbox_inches="tight")
    print(
        f"Figure with SPI in time for the {voi} voivodeship saved in results/SPI_in_time_{voi}.png"
    )
    # plt.show()

In [ ]:
def voi_SPI_pipeline(preprocessed_voi_df, voi):
    if (
        (not os.path.exists(f"../results/{voi}_SPI_monthly.csv"))
        or (not os.path.exists(f"../results/{voi}_SPI_quarterly.csv"))
        or (not os.path.exists(f"../results/{voi}_SPI_yearly.csv"))
    ):
        SPI_1, SPI_3, SPI_12 = get_SPI(
            preprocessed_voi_df,
            voi,
            True,
        )
    else:
        SPI_1 = pd.read_csv(
            f"../results/{voi}_SPI_yearly.csv", index_col="date", parse_dates=True
        )
        SPI_3 = pd.read_csv(
            f"../results/{voi}_SPI_quarterly.csv", index_col="date", parse_dates=True
        )
        SPI_12 = pd.read_csv(
            f"../results/{voi}_SPI_monthly.csv", index_col="date", parse_dates=True
        )
    SPI_1["SPI"] = SPI_1["SPI"].round(2)
    SPI_3["SPI"] = SPI_3["SPI"].round(2)
    SPI_12["SPI"] = SPI_12["SPI"].round(2)
    get_voi_SPI_statistics(SPI_1, SPI_3, SPI_12, voi)
    visualize_voi_SPI(SPI_1, SPI_3, SPI_12, voi)

In [ ]:
# voi_SPI_pipeline(preprocessed_voi_df, "Lubusz")